In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.DEBUG)

In [ ]:
def f(x: int) -> tuple[float, ...]:
    pass

In [ ]:
tuple([1, 2])

In [ ]:
from tsdm.utils.decorators import named_return

In [ ]:
@named_return
def example_func(a: int, b: int) -> tuple[int, int]:
    c = a > 2
    r = a + b
    s = a - b
    if c:
        return r, s
    else:
        return r, s

In [ ]:
example_func(1, 2)

In [ ]:
import ast
import inspect


def collect_exit_points(func):
    tree = ast.parse(inspect.getsource(func))
    exit_points = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Return):
            exit_points.append(node)
    return exit_points


def exit_point_names(func):
    exit_points = collect_exit_points(func)

    var_names = []
    for exit_point in exit_points:
        assert isinstance(exit_point.value, ast.Tuple)

        e = ()
        for obj in exit_point.value.elts:
            assert isinstance(obj, ast.Name)
            e += (obj.id,)
        var_names.append(e)
    return var_names


def example_func(a: int, b: int) -> int:
    c = a > 2
    r = a + b
    s = a - b
    if c:
        return r, s
    else:
        return r, s


returns = collect_exit_points(example_func)

In [ ]:
example_func.__qualname__

In [ ]:
set(exit_point_names(example_func))

In [ ]:
example_func.__code__.co_varnames

In [ ]:
returns[0].value.dims

In [ ]:
import ast


def function_to_ast(func):
    return ast.parse(inspect.getsource(func))

In [ ]:
ast.parse(inspect.getsource(example_func))

In [ ]:
r.__args__

In [ ]:
r = f.__annotations__["return"]

In [ ]:
dir(f.__code__)

In [ ]:
Ellipsis in r.__args__

In [ ]:
issubclass(r.__origin__, tuple)

In [ ]:
dir(int | float)

In [ ]:
dirr

In [ ]:
from tsdm.utils import LazyDict

In [ ]:
d = LazyDict({1: 1})
d.get(1)

In [ ]:
d

In [ ]:
import os
import zipfile
import requests
from pathlib import Path

In [ ]:
import tsdm

In [ ]:
from tsdm.models.pretrained import LinODEnet

In [ ]:
d = {}
?d.get

In [ ]:
LinODEnet.available_checkpoints()

In [ ]:
LinODEnet.get_checkpoints()

In [ ]:
tsdm.utils._utils.repackage_zip("20.zip")

In [ ]:
Path("30.zip").rename("20.zip")

In [ ]:
zf = zipfile.ZipFile("20.zip")
zf

In [ ]:
p = zf.infolist()[1]

In [ ]:
Path("asfagfs/2022-12-01-20.zip").stem

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/P7SAkkaeGtAWJ2L/download?path=/LinODEnet/2022-12-01/20"
local_filename = "2022-12-01-20.zip"

r = requests.get(url, stream=True)

with open(local_filename, "wb") as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:  # filter out keep-alive new chunks
            f.write(chunk)

with zipfile.ZipFile(local_filename, "r") as zip_ref:
    zip_ref.extractall(".")

In [ ]:
import tsdm

In [ ]:
cls = tsdm.models.pretrained.LinODEnet

In [ ]:
pretrained = cls.from_remote_checkpoint("2022-12-01/20")

In [ ]:
pretrained.rawdata_path

In [ ]:
# cls.available_checkpoints()
model = cls.from_path("2022-12-01-linodenet-50-f79c5e885f2182835e7b88ba3205fb33.zip")
model.components["model"]

In [ ]:
pretrained = cls.from_remote_checkpoint("2022-12-01/30")

In [ ]:
model.components["model"]

In [ ]:
import os

In [ ]:
os.path.isdir("2022-12-01-linodenet-50-f79c5e885f2182835e7b88ba3205fb33.zip")

In [ ]:
str(model.rawdata_path)

In [ ]:
model.components["model"]